# Exercise 3a

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Load dataset

In [16]:
trainset = torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

### Model

In [17]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 4 * 4, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

        #self.feature_map = []

    def forward(self, x, fully_connected):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        #self.feature_map = x
        x = torch.flatten(x, 1)
        if fully_connected == 1:
            x = F.leaky_relu(self.fc1(x))
            #x = F.leaky_relu(self.fc2(x))
            x = self.fc2(x)

        return x
model = Net()

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.000001)

writer = SummaryWriter()

### Training

In [18]:
best_accuracy = 0
best_net = 0

for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs, 1)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train 1", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels

        outputs = model(inputs, 1)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train 1", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [12000/12000] - Loss: 2.2158010005950928, accuracy:  0.36025
 (new best)
Finished Training


### Test

In [19]:
correct = 0
total = 0
for i, data in enumerate(test_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs, 1)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.3437


In [31]:
data, labels = next(iter(train_loader))
out = model(inputs, 0)



tensor([[-3.6058e-04,  5.3648e-02,  7.4693e-01,  5.0857e-01,  1.7716e-02,
         -2.0981e-03,  6.9483e-01,  9.0148e-02, -1.4849e-04,  4.1343e-02,
          7.4992e-01, -6.9993e-04, -4.7023e-04,  1.7200e-01,  8.9396e-01,
         -2.4341e-03,  3.3136e-01,  4.4328e-01,  4.2572e-01,  4.0186e-01,
          3.8257e-01,  4.9663e-01,  2.8083e-01,  3.1433e-01,  5.4716e-01,
          2.4557e-01,  4.1934e-01,  4.1658e-01,  5.4916e-01,  4.4743e-01,
          3.0911e-01,  5.4615e-01,  4.5199e-02,  4.7579e-01,  3.4794e-01,
          5.6238e-01,  2.6141e-01,  4.8746e-01,  7.7209e-01,  4.9273e-01,
          3.0482e-01,  4.5822e-01,  7.2758e-01,  4.3915e-01,  4.6673e-01,
          3.1089e-01,  6.9802e-01,  4.8976e-01, -1.4121e-03, -1.8047e-03,
         -3.3777e-03,  4.9360e-01, -1.1944e-03, -3.1888e-03,  1.9287e-01,
          6.3381e-01, -5.4752e-04, -7.4343e-03,  5.7672e-01,  5.1188e-01,
         -1.0669e-03, -7.0064e-03,  7.1688e-01,  2.6511e-01,  3.1824e-02,
          5.1699e-01,  6.6361e-01,  7.

### PCA

In [30]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(out.detach().numpy())

ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'